# Notebook 04 - Regimes & Signals

Translate macro dynamics into interpretable regimes that can later condition decisions — not predict markets.

This notebook:
- formalizes intuition from Notebook 03
- introduces regime logic
- produces labels used by every downstream notebook

In [1]:
# imports

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

from macro_utils.transforms import zscore
from macro_utils.regimes import (
    classify_growth,
    classify_inflation,
    classify_policy,
    combine_regimes,
    build_macro_regimes
)
from macro_utils.utils import build_transformed_dataset, prepare_plot_df

DATA_RAW = PROJECT_ROOT / "data" / "raw"

In [2]:
# load data

monthly = pd.read_csv(
    DATA_RAW / "fred_monthly.csv",
    index_col=0,
    parse_dates=True
)

In [3]:
# transformed inputs

df = build_transformed_dataset(monthly)
df.head()

c:\Users\JR\OneDrive\Mini PC\FRED\macro_utils\transforms.py:5: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  return series.pct_change(periods) * 100
c:\Users\JR\OneDrive\Mini PC\FRED\macro_utils\transforms.py:8: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  return series.pct_change(1) * 100


,GDP_YoY,CPI_YoY,CPI_MoM,UNRATE,FEDFUNDS
DATE,,,,,
1991-01-31,2.766649,5.647059,0.372578,6.4,6.91
1991-02-28,2.766649,5.312500,0.074239,6.6,6.25
1991-03-31,2.766649,4.821151,0.000000,6.8,6.12
1991-04-30,2.799215,4.809930,0.222552,6.7,5.91
1991-05-31,2.799215,5.034857,0.370096,6.9,5.78


Macro regimes provide structure without prediction.

They allow us to answer questions like:
- What environment are we in?
- How do relationships differ across states?
- How do strategies behave conditionally?

Regimes are descriptive, not directional.

In [4]:
df = build_macro_regimes(df)
df[["Growth_Regime", "Inflation_Regime", "Policy_Regime", "Macro_Regime"]].head()

,Growth_Regime,Inflation_Regime,Policy_Regime,Macro_Regime
DATE,,,,
1991-01-31,Expansion,Inflationary,Easing,Expansion / Inflationary / Easing
1991-02-28,Expansion,Inflationary,Easing,Expansion / Inflationary / Easing
1991-03-31,Expansion,Inflationary,Easing,Expansion / Inflationary / Easing
1991-04-30,Expansion,Inflationary,Easing,Expansion / Inflationary / Easing
1991-05-31,Expansion,Inflationary,Easing,Expansion / Inflationary / Easing


In [5]:
plot_df = prepare_plot_df(df)

In [6]:
# Regime counts

df["Macro_Regime"].value_counts()

Macro_Regime
Expansion / Inflationary / Easing           112
Expansion / Inflationary / Tightening       108
Expansion / Disinflationary / Tightening    103
Expansion / Disinflationary / Easing         78
Contraction / Disinflationary / Easing       17
Contraction / Inflationary / Easing           1
Name: count, dtype: int64

In [7]:
# Regime Timeline

fig = px.scatter(
    plot_df,
    x="DATE",
    y="Macro_Regime",
    color="Macro_Regime",
    title="Macro Regime Timeline",
    height=350
)

fig.update_yaxes(showticklabels=False)
fig.show()

In [8]:
# GDP Growth By Regime

fig = px.box(
    plot_df,
    x="Macro_Regime",
    y="GDP_YoY",
    points="outliers",
    title="GDP YoY by Macro Regime",
)

fig.update_layout(height=450)
fig.show()

In [9]:
# Inflation by Regime

fig = px.box(
    plot_df,
    x="Macro_Regime",
    y="CPI_YoY",
    points="outliers",
    title="Inflation (CPI YoY) by Macro Regime",
)

fig.update_layout(height=450)
fig.show()

In [10]:
# Policy Rate by Regime

fig = px.box(
    plot_df,
    x="Macro_Regime",
    y="FEDFUNDS",
    points="outliers",
    title="Fed Funds Rate by Macro Regime",
)

fig.update_layout(height=450)
fig.show()

In [11]:
# Resume Summary Table

summary = (
    df
    .groupby("Macro_Regime")
    .agg({
        "GDP_YoY": ["mean", "std"],
        "CPI_YoY": ["mean", "std"],
        "UNRATE": ["mean", "std"],
        "FEDFUNDS": ["mean", "std"],
    })
)

summary

GDP_YoY             CPI_YoY  \
                                              mean       std      mean   
Macro_Regime                                                             
Contraction / Disinflationary / Easing   -2.722058  2.245557 -0.130521   
Contraction / Inflationary / Easing      -0.726120       NaN  3.731058   
Expansion / Disinflationary / Easing      4.281563  1.625839  1.749719   
Expansion / Disinflationary / Tightening  4.458728  1.084269  1.602413   
Expansion / Inflationary / Easing         5.042124  2.433090  3.524492   
Expansion / Inflationary / Tightening     6.582467  2.066896  3.717800   

                                                      UNRATE            \
                                               std      mean       std   
Macro_Regime                                                             
Contraction / Disinflationary / Easing    1.035789  9.476471  2.032095   
Contraction / Inflationary / Easing            NaN  6.500000       NaN   
Expansion / Disinflationary / Easing      0.459090  5.796154  1.480299   
Expansion / Disinflationary / Tightening  0.688677  5.584466  1.772765   
Expansion / Inflationary / Easing         1.110600  6.123214  1.580739   
Expansion / Inflationary / Tightening     1.573232  4.618519  0.857746   

                                          FEDFUNDS            
                                              mean       std  
Macro_Regime                                                  
Contraction / Disinflationary / Easing    0.151176  0.080147  
Contraction / Inflationary / Easing       0.970000       NaN  
Expansion / Disinflationary / Easing      1.897692  1.900108  
Expansion / Disinflationary / Tightening  1.754078  2.037167  
Expansion / Inflationary / Easing         3.161071  2.011209  
Expansion / Inflationary / Tightening     4.224907  1.799680

In [12]:
# Regime Transition Dectection

from macro_utils.regimes import detect_transitions

transitions = detect_transitions(df["Macro_Regime"])
transitions.head(10)

,date,from,to
0,1993-12-31,Expansion / Inflationary / Easing,Expansion / Inflationary / Tightening
1,1994-01-31,Expansion / Inflationary / Tightening,Expansion / Disinflationary / Tightening
2,1994-02-28,Expansion / Disinflationary / Tightening,Expansion / Inflationary / Tightening
3,1994-04-30,Expansion / Inflationary / Tightening,Expansion / Disinflationary / Tightening
4,1994-07-31,Expansion / Disinflationary / Tightening,Expansion / Inflationary / Tightening
5,1996-02-29,Expansion / Inflationary / Tightening,Expansion / Inflationary / Easing
6,1997-03-31,Expansion / Inflationary / Easing,Expansion / Inflationary / Tightening
7,1997-04-30,Expansion / Inflationary / Tightening,Expansion / Disinflationary / Tightening
8,1998-04-30,Expansion / Disinflationary / Tightening,Expansion / Disinflationary / Easing
9,1998-07-31,Expansion / Disinflationary / Easing,Expansion / Disinflationary / Tightening


In [13]:
# Transition Counts

transitions["to"].value_counts()

to
Expansion / Disinflationary / Easing        16
Expansion / Disinflationary / Tightening    15
Expansion / Inflationary / Easing           15
Expansion / Inflationary / Tightening       13
Contraction / Disinflationary / Easing       2
Contraction / Inflationary / Easing          1
Name: count, dtype: int64

In [14]:
# Transition Timeline

fig = px.scatter(
    transitions,
    x="date",
    y="to",
    color="to",
    title="Macro Regime Transitions",
    height=350
)

fig.update_yaxes(showticklabels=False)
fig.show()